In [1]:
from ashta_chamma_env import AshtaChammaEnv

In [ ]:
a = AshtaChammaEnv()
# a.render()

: 

In [3]:
print([(i // 4, i % 4) for i in range(16)])

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


In [ ]:
import pygame
import time

from helper import diceRoll, move, checkEnemy, possibleMoves, board_overview, adjustDisplay
from player import Player

#---------------------------------------------------------------------------------------------------
pygame.init()
SIZE=(width,height)=(800,600)
screen=pygame.display.set_mode((width,height))

#---------------------------------------------------------------------------------------------------
# Caption & icon
pygame.display.set_caption("Ashta Chamma")
icon=pygame.image.load("assets/icons/game_icon.png")
pygame.display.set_icon(icon)

#-------------------------------------------------------------------------
# Colours
WHITE = (255, 255, 255)
GRAY  = (180, 180, 180)
BLACK = (0, 0, 0)

# Turn colours R->B->Y->G
colors=[(250, 39, 55), (42, 122, 218), (58, 154, 26), (220, 160, 20)]

home_colors=[WHITE, WHITE, WHITE, WHITE]
#---------------------------------------------------------------------------------------------------
# Fonts
OPEN_SANS = "assets/fonts/OpenSans-Regular.ttf"
smallFont = pygame.font.Font(OPEN_SANS, 20)
mediumFont = pygame.font.Font(OPEN_SANS, 28)
largeFont = pygame.font.Font(OPEN_SANS, 40)


#---------------------------------------------------------------------------------------------------
# Board setup
HEIGHT = 9
WIDTH = 9
BOARD_PADDING = 20												  
board_height = ((7/8) * height) - (BOARD_PADDING * 2)	
board_width = width - (BOARD_PADDING * 2)
cell_size = int(min(board_height / WIDTH, board_width / HEIGHT))
board_origin = (BOARD_PADDING, BOARD_PADDING)

#----------------------------------------------------------------------------------------------------------
# Intialization
START=True
isBoardDrawn=True
running=True
changeTurn=True
mainPage=True
helpPage=False
rollDone=False
calculateNow=False
moveDone=True
disNum=""
moves=[]
k=[]
for i in range(4):
	k.append(False)
allPawns=[]

turn=-1
safe_places=[ (1,4), (2,2), (2,6), (4,1), (4,4), (4,7), (6,2), (6,6), (7,4) ]
home_places=[(4,8), (8,4), (4,0), (0,4)]

#----------------------------------------------------------------------------------------
# Importing and Resizing Images
cross=pygame.image.load("assets/icons/cross.png")
cross=pygame.transform.scale(cross,(53,53))

game_name=pygame.image.load("assets/icons/logo.gif")
game_name=pygame.transform.scale(game_name,(600,100))

instructions=pygame.image.load("assets/icons/instruction.png")
instructions=pygame.transform.scale(instructions, (790,350))
def drawBoard():
	""" Draws the board on to the screen and return a 2D list representing each positon of the square """

	cells = []
	h_color=0
	for i in range(HEIGHT):
		row = []
		for j in range(WIDTH):

			# Draw rectangle for cell
			rect = pygame.Rect(
				board_origin[0] + j * cell_size,
				board_origin[1] + i * cell_size,
				cell_size, cell_size
			)
			if(i!=0 and i!=HEIGHT-1 and j!=0 and j!=WIDTH-1):
				if((i,j) in safe_places):
					pygame.draw.rect(screen, (240, 207, 174), rect)
					pygame.draw.rect(screen, WHITE, rect, 2)
					screen.blit(cross,(board_origin[0] + j * cell_size,board_origin[1] + i * cell_size))
				else:
					pygame.draw.rect(screen, (240, 207, 174), rect)
					pygame.draw.rect(screen, WHITE, rect, 2)
			else:
				if((i,j) in home_places):
					pygame.draw.rect(screen,home_colors[h_color], rect)
					pygame.draw.rect(screen, WHITE, rect, 2)
					h_color=(h_color+1)%4
					
				else:
					pygame.draw.rect(screen, BLACK, rect)
					pygame.draw.rect(screen, BLACK, rect, 2)
			row.append(rect)
		cells.append(row)
	return cells
cells = drawBoard()
print(cells)

pygame 2.5.2 (SDL 2.28.3, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
[[<rect(20, 20, 53, 53)>, <rect(73, 20, 53, 53)>, <rect(126, 20, 53, 53)>, <rect(179, 20, 53, 53)>, <rect(232, 20, 53, 53)>, <rect(285, 20, 53, 53)>, <rect(338, 20, 53, 53)>, <rect(391, 20, 53, 53)>, <rect(444, 20, 53, 53)>], [<rect(20, 73, 53, 53)>, <rect(73, 73, 53, 53)>, <rect(126, 73, 53, 53)>, <rect(179, 73, 53, 53)>, <rect(232, 73, 53, 53)>, <rect(285, 73, 53, 53)>, <rect(338, 73, 53, 53)>, <rect(391, 73, 53, 53)>, <rect(444, 73, 53, 53)>], [<rect(20, 126, 53, 53)>, <rect(73, 126, 53, 53)>, <rect(126, 126, 53, 53)>, <rect(179, 126, 53, 53)>, <rect(232, 126, 53, 53)>, <rect(285, 126, 53, 53)>, <rect(338, 126, 53, 53)>, <rect(391, 126, 53, 53)>, <rect(444, 126, 53, 53)>], [<rect(20, 179, 53, 53)>, <rect(73, 179, 53, 53)>, <rect(126, 179, 53, 53)>, <rect(179, 179, 53, 53)>, <rect(232, 179, 53, 53)>, <rect(285, 179, 53, 53)>, <rect(338, 179, 53, 53)>, <rect(391, 179, 53, 53

: 

In [ ]:
from board import Board
from player import Player
# Create the board instance
board = Board()

# Add players
board.add_player(Player(0, [(0,4), (0,4), (0,4), (0,4)], (250, 39, 55)))  # Red player
board.add_player(Player(1, [(4,0), (4,0), (4,0), (4,0)], (42, 122, 218)))  # Blue player
board.add_player(Player(2, [(8,4), (8,4), (8,4), (8,4)], (0, 128, 0)))
board.add_player(Player(3, [(4,8), (4,8), (4,8), (4,8)], (255, 255, 0)))
# Get possible moves for player 0
possible_moves = board.move(0)
possible_moves = board.move(1)
possible_moves = board.move(2)
possible_moves = board.move(3)
print("Possible Moves for Player 0:", possible_moves)

# Update the board with a move
# if possible_moves:
#     pawn_index, new_position = possible_moves[0]
#     board.update(board.board_state, 0, pawn_index, new_position)

print(board.players)
board.render()

pygame 2.5.2 (SDL 2.28.3, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Possible Moves for Player 0: [(4, 8), (4, 8), (4, 8), (2, 7)]
[<player.Player object at 0x000002642EEF9280>, <player.Player object at 0x000002642EEF9220>, <player.Player object at 0x000002642EEF92B0>, <player.Player object at 0x000002642F086070>]


: 

In [61]:
print(board.players[0].pawns)

[(3, 4), (3, 4), (3, 4), (4, 4)]


In [68]:
possible_moves = board.move(0)
print("Possible Moves for Player 0:", possible_moves)

Possible Moves for Player 0: [(4, 4), (3, 4), (3, 4), (4, 4)]


: 

In [5]:
import random
print(random.randint(0,3))

3
